**NLP Homework 1**
*by Kharkhanova Ekaterina*





Используем данные из готового датасета с отзывами на мобильные телефоны

In [ ]:
import pandas as pd
import nltk

df = pd.read_csv('/content/data.csv', error_bad_lines=False, engine="python")
df.head(100)

,Review,Rating
0,3D Touch просто восхитительная вещь! Заряд дер...,5
1,"Отключается при температуре близкой к нулю, не...",4
2,"В Apple окончательно решили не заморачиваться,...",3
3,Постарался наиболее ёмко и коротко описать все...,4
4,Достойный телефон. Пользоваться одно удовольст...,5
...,...,...
95,"Ну что сказать)? Кому то по душе ios, кому то ...",4
96,Судите сами. После iPhone 4s взял sony Z1 comp...,5
97,Мой путь смартфонопользования начинался с винд...,5
98,"отличный телефон, больше всего радует стабильн...",5


Создаем тестовую выборку, чтобы использовать ее позже

In [ ]:
positive_check_base = df[df.Rating==5].head(5)
positive_check_base

,Review,Rating
0,3D Touch просто восхитительная вещь! Заряд дер...,5
4,Достойный телефон. Пользоваться одно удовольст...,5
5,6s gold 64gb,5
6,Мой первый айфон. Скажу честно- Эппл ранее ник...,5
7,Мне очень понравилась эта модель! Во-первых кл...,5


In [ ]:
negative_check_base = df[df.Rating==1].head(5)
negative_check_base

,Review,Rating
38,Очень сильно разочарован компанией SONY. Испор...,1
51,"Дизайн , конечно, хорош.",1
56,В первый год эксплуатации еще было все нормаль...,1
60,Зарекаюсь от продукции Асус. Смартфон невероят...,1
62,"В первые 2 недели все работало, спустя это вре...",1


In [ ]:
positive_check_base = positive_check_base['Review'].tolist()
negative_check_base = negative_check_base['Review'].tolist()
check_base = positive_check_base + negative_check_base
check_base
right_answers = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]


Создаем обучающую выборку

In [ ]:
positive_train_base = df[df.Rating==5].tail(40)
positive_train_base = positive_train_base['Review'].tolist()
positive_train_base = ' '.join(positive_train_base)
positive_train_base

'после покупки сразу зарядите, как написано в инструкции, 10 часов! и заряжайте желательно через зарядное, а не от компа. тогда аккум будет в норме. для энергосбережения ставлю ручной режим поиска сети, чтоб не делал бесполезных попыток регистрации в неродных сетях (ведь регистрации в сети очень энергоемкая вещь). чтобы не вздувался аккумулятор, нужно: 1. не советую заряжать от компа. 2. ставьте на зарядку девайс только тогда, когда ему это понадобится. 3. снимаете с зарядки соответственно только тогда, когда зарядится.  следуя этим важным правилам у вас аккумулятор прослужит немного дольше. отличная модель не имеющая конкурентов по цене. советую всем читают все форматы ява игр у меня ещё и асус р535 намного дороже, а разница небольшая (нет gps и по мелочи...)зайдите на официальный форум по роверам и увидите множество хороших отзывов и множество решений на ваши возникающие вопросы(например о батарее). я купил и не жалею... конечно не так удобен как обычный телефон или ай-фон. вообще за

In [ ]:
negative_train_base = df[df.Rating==1].tail(40)
negative_train_base = negative_train_base['Review'].tolist()
negative_train_base = ' '.join(negative_train_base)
negative_train_base

'ну что сказать.. четыре гига памяти, невозможность установить свою мелодию на звонок (mp3 в жанре ringtone категорически не хотят синхронизироваться с телефоном) и практически отсутствие блютуза - это, конечно, круто для телефона за такую цену. zune - это вообще отдельная песня. кто пользовался, тот поймет. тяжелый случай клинического идиотизма компании microsoft. в недостатках купила windows phone очень пожалела. до него был       samsung wave 525 и то лучше. как накопитель использовать невозможно, т. к. памяти мало, а отдела для карты памяти нет. ужасная модель. телефон сырой на 100% категорически не рекомендую. до этого был другой leagoo. я его сам утопил. а так продолжал бы пользоваться. но конкретно эта модель жутко глючная. обходите её стороной. лучше возьмите простенький самсунг за эти деньги или подсобирайте еще. очень жалею что скупился и не добавил пару тройку тысяч на тот же самсунг,жаль категорически не рекомендую. до этого был другой leagoo. я его сам утопил. а так продол

Preprocessing

In [ ]:
!pip uninstall spacy

Found existing installation: spacy 2.1.0
Uninstalling spacy-2.1.0:
  Would remove:
    /usr/local/bin/spacy
    /usr/local/lib/python3.7/dist-packages/spacy-2.1.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/spacy/*
Proceed (y/n)? y
  Successfully uninstalled spacy-2.1.0


In [ ]:
!pip install spacy==3.1

     |████████████████████████████████| 6.4 MB 4.6 MB/s 
     |████████████████████████████████| 126 kB 55.2 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 456 kB 55.6 MB/s 
     |████████████████████████████████| 9.8 MB 26.4 MB/s 
     |████████████████████████████████| 10.1 MB 31.4 MB/s 
     |████████████████████████████████| 623 kB 49.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: preshed
    Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Attempting uninstall: blis
    Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninsta

In [ ]:
from spacy.lang.ru import Russian

In [ ]:
nlp = Russian()
doc_p = nlp(positive_train_base.lower())
token_list_p = []
for token in doc_p:
  if token.is_alpha or token.like_num or not token.is_punct:
    token_list_p.append(token.lemma_)

positive_tokens = set(token_list_p)


In [ ]:
doc_n = nlp(negative_train_base.lower())
token_list_n = []
for token in doc_n:
  if token.is_alpha or token.like_num or not token.is_punct:
    token_list_n.append(token.lemma_)

negative_tokens = set(token_list_n)


Получаем множества со словами только из положительных и только из отрицательных отзывов:

In [ ]:
positive_set = positive_tokens.difference(negative_tokens)
negative_set = negative_tokens.difference(positive_tokens)

**Работа с тестовой выборкой**

In [ ]:
def model_checker(test_base):
  answers = []
  for text in test_base:
    doc_t = nlp(text.lower())
    token_list_t = []
    for token in doc_t:
      if token.is_alpha or token.like_num or not token.is_punct:
        token_list_n.append(token.lemma_)

    tokens = set(token_list_t)
    positivity = len(tokens.intersection(positive_set))
    negativity = len(tokens.intersection(negative_set))
    if positivity == 0 and negativity == 0:
      answers.append(2)
    if positivity > negativity:
      answers.append(1)
    if positivity < negativity:
      answers.append(0)

  return answers

model_checker(check_base)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

accuracy

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model_checker(check_base)
y_true = right_answers
accuracy_score(y_true, y_pred)

0.0

Да, accuracy, к сожалению, нулевая, но в целом, программа должна работать. Тестовую выборку функция распознает как нейтральную по следующим причинам: **Варианты улучшения программы**
1) на этапе предобработки обучающей выборки были проблемы с лемматизацией, она полностью корректно сработала только на глаголах. Библиотеки, с помощью которых я пыталась лемматизировать текст (еще одна русскоязычная модель spacy, nltk, natasha), не совмещались между собой или не работали у меня на компьютере.
2) при создании множеств негативных и положительных слов можно было использовать параметр частоты слов, чтобы из множеств не исключались часто встречающиеся в одной и редко в другой слова. Цифры были оставлены специально, тк они могут свидетельствовать об объеме памяти или модели устройства, о которой у большой группы авторов отзывов сложилось хорошее мнение